In [ ]:
import spacy
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import torch
import string
import matplotlib.pyplot as plt
from transformers import AutoModel, AutoTokenizer
import pandas as pd
import zipfile
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Path to the gzip file in your Google Drive
PATH_TO_NOTEEVENTS_GZ = '/content/drive/My Drive/NOTEEVENTS.csv.gz'  # Edit to the correct path
PATH_TO_NOTEEVENTS_CSV = '/content/NOTEEVENTS.csv'  # Path to extract the CSV file to

# Read the gzipped CSV file directly using pandas
df_noteevents = pd.read_csv(PATH_TO_NOTEEVENTS_GZ, compression='gzip')

<ipython-input-4-92e88c351025>:6: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df_noteevents = pd.read_csv(PATH_TO_NOTEEVENTS_GZ, compression='gzip')


In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

!gcloud projects list

from google.cloud import bigquery

# Construct a BigQuery client object.
client = bigquery.Client(project='clinical-entity-extraction')

query = """
    SELECT * FROM `physionet-data.mimiciii_clinical.d_icd_diagnoses`
"""
query_job = client.query(query)
d_icd_diagnoses = query_job.to_dataframe()

#table for diagnostic icd
query = """
    SELECT * FROM `physionet-data.mimiciii_clinical.diagnoses_icd`
"""
query_job = client.query(query)
diagnoses_icd = query_job.to_dataframe()

Authenticated
PROJECT_ID                  NAME                        PROJECT_NUMBER
clinical-entity-extraction  clinical-entity-extraction  321960627270
dulcet-elevator-257302      Personal API project        731274068268
hackhelper-cb7fe            HackOrganized               284511353607
physionet-data-449900       physionet-data              398801384102
ricehack-ac1a8              riceHack                    1054752105505
safesound-e5d17             safesound                   14104572424
stayalert-8b5b2             StayAlert                   721751451925


In [ ]:
# Get the top 10 most frequent ICD-9 codes and their counts
top_icd_codes = diagnoses_icd["ICD9_CODE"].value_counts().nlargest(10).reset_index()
top_icd_codes.columns = ["ICD9_CODE", "Count"]

# Filter d_icd_diagnoses only once for the relevant ICD-9 codes
icd_titles = d_icd_diagnoses.loc[d_icd_diagnoses["ICD9_CODE"].isin(top_icd_codes["ICD9_CODE"]),
                                 ["ICD9_CODE", "LONG_TITLE"]].drop_duplicates()

# Merge count data with long titles efficiently
df_top_icd = top_icd_codes.merge(icd_titles, on="ICD9_CODE", how="left")

# Display the final DataFrame
print(df_top_icd)

  ICD9_CODE  Count                                         LONG_TITLE
0      4019  20703                 Unspecified essential hypertension
1      4280  13111              Congestive heart failure, unspecified
2     42731  12891                                Atrial fibrillation
3     41401  12429  Coronary atherosclerosis of native coronary ar...
4      5849   9119                  Acute kidney failure, unspecified
5     25000   9058  Diabetes mellitus without mention of complicat...
6      2724   8690               Other and unspecified hyperlipidemia
7     51881   7497                          Acute respiratory failure
8      5990   6555        Urinary tract infection, site not specified
9     53081   6326                                  Esophageal reflux


In [ ]:
## Filter rows with ICD9 == 4280   #Congestive heart failure, unspecified
ICD_FILTER = '4280'
df_filtered = diagnoses_icd[diagnoses_icd['ICD9_CODE'] == ICD_FILTER]
subj_filtered = df_filtered['SUBJECT_ID'].unique()
print(f"Number of ICD-9 {ICD_FILTER} patients: {subj_filtered.size}")
df_noteevents_filtered = df_noteevents[df_noteevents['SUBJECT_ID'].isin(subj_filtered)]
df_noteevents_filtered.info()

Number of ICD-9 4280 patients: 9843
<class 'pandas.core.frame.DataFrame'>
Index: 640843 entries, 4 to 2066679
Data columns (total 11 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   ROW_ID       640843 non-null  int64  
 1   SUBJECT_ID   640843 non-null  int64  
 2   HADM_ID      550153 non-null  float64
 3   CHARTDATE    640843 non-null  object 
 4   CHARTTIME    501803 non-null  object 
 5   STORETIME    338651 non-null  object 
 6   CATEGORY     640843 non-null  object 
 7   DESCRIPTION  640843 non-null  object 
 8   CGID         338651 non-null  float64
 9   ISERROR      354 non-null     float64
 10  TEXT         640843 non-null  object 
dtypes: float64(3), int64(2), object(6)
memory usage: 58.7+ MB


In [ ]:
#df_noteevents_filtered['CATEGORY'].unique()
category_counts = df_noteevents_filtered['CATEGORY'].value_counts()

# Display the counts
print(category_counts)

CATEGORY
Nursing/other        179081
Radiology            163989
ECG                   99022
Nursing               83655
Physician             54864
Echo                  20751
Discharge summary     18430
Respiratory           11097
General                3432
Nutrition              3430
Rehab Services         1889
Social Work             710
Case Management         418
Consult                  55
Pharmacy                 20
Name: count, dtype: int64


In [ ]:
discharge_summary_texts = df_noteevents_filtered[df_noteevents_filtered['CATEGORY'] == 'Discharge summary']["TEXT"].dropna()

# Print or process the list
print(discharge_summary_texts.iloc[0])

Admission Date:  [**2162-3-3**]              Discharge Date:   [**2162-3-25**]

Date of Birth:  [**2080-1-4**]             Sex:   M

Service: MEDICINE

Allergies:
Patient recorded as having No Known Allergies to Drugs

Attending:[**First Name3 (LF) 1828**]
Chief Complaint:
Mr. [**Known lastname 1829**] was seen at [**Hospital1 18**] after a mechanical fall from
a height of 10 feet.  CT scan noted unstable fracture of C6-7 &
posterior elements.

Major Surgical or Invasive Procedure:
1. Anterior cervical osteotomy, C6-C7, with decompression and
excision of ossification of the posterior longitudinal ligament.
2. Anterior cervical deformity correction.
3. Interbody reconstruction.
4. Anterior cervical fusion, C5-C6-C7.
5. Plate instrumentation, C5-C6-C7.
6. Cervical laminectomy C6-C7, T1.
7. Posterior cervical arthrodesis C4-T1.
8. Cervical instrumentation C4-T1.
9. Arthrodesis augmentation with autograft, allograft and
demineralized bone matrix.


History of Present Illness:
Mr. [**Known 

In [ ]:
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.1/en_ner_bc5cdr_md-0.5.1.tar.gz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.2/120.2 MB 7.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.9/153.9 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 90.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 108.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 917.4/917.4 kB 60.3 MB/s eta 0:00:00
  Created wheel for en_ner_bc5cdr_md: filename=en_ner_bc5cdr_md-0.5.1-py3-none-any.whl size=120217626 sha256=746a4540d57613d18d277998ca52c23af02151f2c0992575d727c0fc8a66f910
  Stored in directory: /root/.cache/pip/wheels/b4/8d/fb/36403b97e0de3d4c3f81c95f9d22c423d90f542243a35d5350
Successfully built en_ner_bc5cdr_md
  Attempting uninstall: wasabi
    Found existing installation: wasabi 1.1.3
    Unins

In [ ]:
nlp_scispacy = spacy.load("en_ner_bc5cdr_md")  # Clinical NLP model

/usr/local/lib/python3.11/dist-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_ner_bc5cdr_md' (0.5.1) was trained with spaCy v3.4.1 and may not be 100% compatible with the current version (3.7.5). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  


In [ ]:
from spacy import displacy
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

In [ ]:
# Load PEGASUS model for medical summarization
model_name = "google/pegasus-pubmed"  # Best for summarizing medical texts
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-pubmed and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

In [ ]:
# Function to clean text (removes newlines, special characters, and extra spaces)
def clean_text(text):
    text = text.replace("\n", " ").replace("\t", " ")  # Remove newlines/tabs
    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with a single space
    text = re.sub(r'[^a-zA-Z0-9., ]', '', text)  # Remove special characters
    return text.strip()  # Trim leading and trailing spaces

In [ ]:
import nltk
nltk.download("punkt_tab")

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
def extract_named_entities(text, nlp_scispacy):
    """
    Extracts named entities from text using SciSpacy NLP model.
    """
    doc = nlp_scispacy(text)
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    return entities, doc

def split_long_sentences(text, tokenizer: PreTrainedTokenizer, max_tokens_per_sentence=250):
    """
    Splits overly long sentences only if necessary to maintain coherence.
    """
    sentences = nltk.tokenize.sent_tokenize(text)
    fixed_sentences = []

    for sentence in sentences:
        tokens = tokenizer.encode(sentence, add_special_tokens=False)

        if len(tokens) > max_tokens_per_sentence and "," in sentence:
            sub_sentences = sentence.split(", ")
            for sub_sentence in sub_sentences:
                fixed_sentences.append(sub_sentence.strip())
        else:
            fixed_sentences.append(sentence)

    return fixed_sentences

def chunk_text(text, tokenizer: PreTrainedTokenizer, chunk_size=1024, overlap=3):
    """
    Splits text into properly sized chunks while ensuring token limits.
    """
    sentences = split_long_sentences(text, tokenizer)
    chunks = []
    current_chunk = []
    current_length = 0

    print(f"Total Sentences in Input: {len(sentences)}")
    print(f"Total Tokens in Input: {len(tokenizer.encode(text, add_special_tokens=False))}")

    for i, sentence in enumerate(sentences):
        sent_tokens = tokenizer.encode(sentence, add_special_tokens=False)

        if current_length + len(sent_tokens) > chunk_size:
            if current_chunk:
                chunks.append(" ".join(current_chunk))
                print(f"Chunk {len(chunks)} created with {current_length} tokens")

            current_chunk = current_chunk[-overlap:] if len(current_chunk) > overlap else []
            current_length = sum(len(tokenizer.encode(sent, add_special_tokens=False)) for sent in current_chunk)

        current_chunk.append(sentence)
        current_length += len(sent_tokens)

    if current_chunk:
        chunks.append(" ".join(current_chunk))
        print(f"Final Chunk {len(chunks)} created with {current_length} tokens")

    print(f"Total Chunks Created: {len(chunks)}")
    return chunks

def generate_summary(text, tokenizer: PreTrainedTokenizer, model: PreTrainedModel, max_length=150):
    """
    Generates a summary for a given text chunk using a provided model and tokenizer.
    """
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=1024)
    summary_ids = model.generate(inputs["input_ids"], max_length=max_length, num_beams=4, early_stopping=True)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

def summarize_text(text, tokenizer: PreTrainedTokenizer, model: PreTrainedModel, chunk_size=1024, overlap=100, summary_length=150):
    """
    Summarizes long text by chunking and summarizing each part individually.
    """
    chunks = chunk_text(text, tokenizer, chunk_size, overlap)
    chunk_summaries = [generate_summary(chunk, tokenizer, model, max_length=summary_length) for chunk in chunks]
    return "\n\n".join(chunk_summaries)

In [ ]:
from transformers import PreTrainedTokenizer, PreTrainedModel

def process_discharge_note(discharge_summary_texts, tokenizer: PreTrainedTokenizer, model: PreTrainedModel, nlp_scispacy):
    """
    Processes a single discharge summary note:
    - Cleans the text
    - Tokenizes and counts tokens
    - Extracts named entities before and after summarization
    - Summarizes the text
    - Displays named entity visualization and tables
    """
    # Select the first discharge note
    first_note = discharge_summary_texts.iloc[0]

    # Tokenize and count tokens before summarization
    tokenized_text = tokenizer(first_note, return_tensors="pt", truncation=False)
    num_tokens_before = tokenized_text["input_ids"].shape[1]
    print(f"Original Token Count: {num_tokens_before}")

    # Clean the text
    cleaned_text = clean_text(first_note)

    # Extract named entities before summarization
    original_entities, original_doc = extract_named_entities(cleaned_text, nlp_scispacy)

    # Summarize the text
    summarized_text = summarize_text(cleaned_text, tokenizer, model)

    # Tokenize and count tokens after summarization
    tokenized_text = tokenizer(summarized_text, return_tensors="pt", truncation=False)
    num_tokens_after = tokenized_text["input_ids"].shape[1]
    print(f"Summarized Token Count: {num_tokens_after}")

    # Extract named entities after summarization
    summarized_entities, summarized_doc = extract_named_entities(summarized_text, nlp_scispacy)

    # Display named entities before summarization
    print("Original Text with Named Entities:")
    displacy.render(original_doc, style="ent", jupyter=True)

    # Display named entities after summarization
    print("Summarized Text with Named Entities:")
    displacy.render(summarized_doc, style="ent", jupyter=True)

    # Create DataFrames for named entities
    df_original_entities = pd.DataFrame(original_entities, columns=["Entity", "Type"])
    df_summarized_entities = pd.DataFrame(summarized_entities, columns=["Entity", "Type"])

    # Display named entities in original text
    print("Named Entities in Original Text:")
    print(df_original_entities)

    # Display named entities in summarized text
    print("Named Entities in Summarized Text:")
    print(df_summarized_entities)

    return {
        "original_text": first_note,
        "summarized_text": summarized_text,
        "original_entities": df_original_entities,
        "summarized_entities": df_summarized_entities,
        "num_tokens_before": num_tokens_before,
        "num_tokens_after": num_tokens_after
    }

In [ ]:
results = process_discharge_note(discharge_summary_texts, tokenizer, model, nlp_scispacy)


Original Token Count: 3324
Total Sentences in Input: 216
Total Tokens in Input: 2965
Chunk 1 created with 1012 tokens
Chunk 2 created with 1023 tokens
Final Chunk 3 created with 930 tokens
Total Chunks Created: 3
Summarized Token Count: 441
Original Text with Named Entities:


Summarized Text with Named Entities:


Named Entities in Original Text:
           Entity      Type
0    Birth 208014   DISEASE
1       Allergies   DISEASE
2       Allergies   DISEASE
3        fracture   DISEASE
4          C5C6C7   DISEASE
..            ...       ...
108     Pneumonia   DISEASE
109           CMO   DISEASE
110      morphine  CHEMICAL
111          pain   DISEASE
112    olanzapine  CHEMICAL

[113 rows x 2 columns]
Named Entities in Summarized Text:
                                 Entity      Type
0                        bladder injury   DISEASE
1                        bladder injury   DISEASE
2                                barium  CHEMICAL
3                                barium  CHEMICAL
4                      encephalomalacia   DISEASE
5                     pneumomediastinum   DISEASE
6                                barium  CHEMICAL
7                      encephalomalacia   DISEASE
8           temporomandibular disorders   DISEASE
9   temporomandibular joint dysfunction   DISEASE
10          temporoman

In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration

In [ ]:
model_name2 = "facebook/bart-large-cnn"  # Use BART for summarization
tokenizer2 = BartTokenizer.from_pretrained(model_name2)
model2 = BartForConditionalGeneration.from_pretrained(model_name2)

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

In [ ]:
results = process_discharge_note(discharge_summary_texts, tokenizer2, model2, nlp_scispacy)

Original Token Count: 4280
Total Sentences in Input: 216
Total Tokens in Input: 3315
Chunk 1 created with 1022 tokens
Chunk 2 created with 1007 tokens
Chunk 3 created with 1020 tokens
Final Chunk 4 created with 301 tokens
Total Chunks Created: 4
Summarized Token Count: 329
Original Text with Named Entities:


Summarized Text with Named Entities:


Named Entities in Original Text:
           Entity      Type
0    Birth 208014   DISEASE
1       Allergies   DISEASE
2       Allergies   DISEASE
3        fracture   DISEASE
4          C5C6C7   DISEASE
..            ...       ...
108     Pneumonia   DISEASE
109           CMO   DISEASE
110      morphine  CHEMICAL
111          pain   DISEASE
112    olanzapine  CHEMICAL

[113 rows x 2 columns]
Named Entities in Summarized Text:
                         Entity      Type
0                      fracture   DISEASE
1                      hematoma   DISEASE
2                       Lucency   DISEASE
3   cerebellar encephalomalacia   DISEASE
4                    infarction   DISEASE
5                        Cspine  CHEMICAL
6                      fracture   DISEASE
7                      fracture   DISEASE
8                        cspine  CHEMICAL
9                   spondylosis   DISEASE
10                calcification   DISEASE
11         Fracture dislocation   DISEASE
12                    Pneu